In [ ]:
import pandas as pd

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import scipy
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizer

def create_dataframe_from_text(file_path):
    # Read the content of the text file
    with open(file_path, 'r') as file:
        content = file.read()

    # Create a DataFrame
    data = {
        "date": ["N/A"],  # Placeholder; you can modify to extract date if present
        "ticker": ["N/A"],  # Placeholder; adjust as needed
        "headline": ["N/A"],  # Placeholder; adjust as needed
        "content": [content]
    }

    df = pd.DataFrame(data)

    # Tokenize the content for BERT
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    df['input_ids'] = df['content'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True, max_length=512))

    return df

file_path = "/content/news2.txt"  # Replace with your text file path
df = create_dataframe_from_text(file_path)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def finbert_sentiment(text: str) -> tuple[float, float, float, str]:
    with torch.no_grad():
        inputs = tokenizer(
            text, return_tensors="pt", padding=True, truncation=True, max_length=512
        )
        outputs = model(**inputs)
        logits = outputs.logits
        scores = {
            k: v
            for k, v in zip(
                model.config.id2label.values(),
                scipy.special.softmax(logits.numpy().squeeze()),
            )
        }
        return (
            scores["positive"],
            scores["negative"],
            scores["neutral"],
            max(scores, key=scores.get),
        )

In [ ]:
# Notice that this is the raw text, no preprocessing
df[["finbert_pos", "finbert_neg", "finbert_neu", "finbert_sentiment"]] = (
    df["content"].apply(finbert_sentiment).apply(pd.Series)
)
df["finbert_score"] = df["finbert_pos"] - df["finbert_neg"]

In [ ]:
df[
    [
        "content",
        "finbert_pos",
        "finbert_neg",
        "finbert_neu",
        "finbert_sentiment",
        "finbert_score",
    ]
]

,content,finbert_pos,finbert_neg,finbert_neu,finbert_sentiment,finbert_score
0,**Mukul Corp Reports Q3 Earnings with Mixed Re...,0.064218,0.818705,0.117077,negative,-0.754487
